In [ ]:
from ipywidgets import HTML

In [ ]:
import requests as req

URL = "https://api.scryfall.com"
HEADERS = {
    'User-Agent': 'MTGPriceChecker/0.1',
    'Accept': '*/*'
}


In [ ]:
def get_card(n):
    return req.get(f"{URL}/cards/blb/{n}").json()

def get_sets():
    return req.get(f"{URL}/sets").json()

In [ ]:
# HTML Components

def set_list(sets):
    return HTML(f"""
        <div class='set-list'>
            {[
                set_card(SET)
                for SET in sets
            ]}
        </div>
    """)

def set_card(set_object):
    return HTML(f"""
        <div>
            <h2>{set_object['name']} ({set_object['code']})</h2>
            {set_object['card_count']} cards, released {set_object['released_at']}
        </div>
    """)

In [ ]:
sets = get_sets()

In [ ]:
set_list(sets['data'][0:5])

In [ ]:
import pandas as pd
import os

In [ ]:
from tqdm import tqdm

def load_set(set_name):
    if not os.path.exists(f"cards_{set_name}.csv"):
        res = req.get(f"{URL}/sets/{set_name}")
        NUMBER_OF_CARDS = res.json()['card_count']

        import time as t
        cards = []


        for n in tqdm(range(1, NUMBER_OF_CARDS)):
            cards.append(get_card(n))
            t.sleep(.15)

        
        params = ['collector_number', 'name', 'image_uris', 'promo', 'rarity', 'prices']
        new_params = ['id', 'name', 'image', 'promo', 'rarity', 'prices']


        CARDS = []
        for card in cards:
            new_card = dict([(np, card[p]) for np, p in zip(new_params, params)])
            new_card['id'] = int(new_card['id'])
            new_card['image'] = new_card['image']['normal']
            new_card['standard_price'] = new_card['prices']['usd']
            new_card['foil_price'] = new_card['prices']['usd_foil']
            del new_card['prices']

            new_card['owned'] = False

            CARDS.append(new_card)

        df = pd.DataFrame.from_records(CARDS)
        df.to_csv("cards.csv", index=False)

        del cards
        del CARDS
        del res
        del NUMBER_OF_CARDS

    df = pd.read_csv(f"cards_{set_name}.csv", index_col="id")
    
    return df

In [ ]:
def save(df):
    df.to_csv("cards.csv")

In [ ]:
df[~df.owned].sort_values("standard_price")

In [ ]:
from ipywidgets import HTML

In [ ]:
display(
    HTML(
        '\n'.join(
            [
                f'<img src="{df.loc[i].image.replace("normal", "small")}">'
                for i in df[~df.owned].sort_values("standard_price").index
            ]
        )
    )
)